In [1]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np

d:\Program\Anaconda\envs\CP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
df_posts.shape

(16361, 19)

In [67]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

In [92]:
# 查找cluster为"Great_Wave_Kanagawa"的帖子
df_out_data = get_post_date_out_range(df_posts,'2023-08-20', '2023-08-31','treatment_japan_waste_nuclear')
print(df_out_data.shape)
# 从df_posts中删除df_1
df_posts = df_posts.drop(df_out_data.index)
print(df_posts.shape)
# 保存


(0, 19)
(15663, 19)


In [93]:
get_post_date_range(df_posts,'2023-08-20', '2023-08-31','treatment_japan_waste_nuclear').shape

(168, 19)

In [94]:
df_posts.to_csv('all_posts.csv',index=False,encoding='utf-8')